In [1]:
import torch
import torch.nn as nn
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetworkGraph
from sinabs.layers import Merge, IAFSqueeze

In [2]:
torch.manual_seed(0)

In [3]:
channels = 1
height = 28
width = 28

input_shape = (channels, height, width)

## Network Module (pure Pytorch)

In [4]:
class SNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(1, 30, 2, 1, bias=False) # node 0
        self.iaf1 = IAFSqueeze(batch_size=1)            # node 1
        self.pool1 = nn.AvgPool2d(2,2)                  # node 2
        self.pool1a = nn.AvgPool2d(4,4)                 # node 3

        self.conv2 = nn.Conv2d(30, 30, 2, 1, bias=False)# node 4
        self.iaf2 = IAFSqueeze(batch_size=1)            # node 6
        self.pool2 = nn.AvgPool2d(2,2)                  # node 7

        self.conv3 = nn.Conv2d(30, 1, 3, 1, bias=False) # node 8
        self.iaf3 = IAFSqueeze(batch_size=1)            # node 9

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(16, 500, bias=False)       # node 10
        self.iaf4 = IAFSqueeze(batch_size=1)            # node 11
        
        self.fc2 = nn.Linear(500, 10, bias=False)       # node 12
        self.iaf5 = IAFSqueeze(batch_size=1)            # node 13

        self.adder = Merge()

    def forward(self, x):
        
        con1_out = self.conv1(x)
        iaf1_out = self.iaf1(con1_out)
        pool1_out = self.pool1(iaf1_out)
        pool1a_out = self.pool1a(iaf1_out)

        conv2_out = self.conv2(pool1_out)
        iaf2_out = self.iaf2(conv2_out)
        pool2_out = self.pool2(iaf2_out)

        conv3_out = self.conv3(self.adder(pool1a_out, pool2_out))
        iaf3_out = self.iaf3(conv3_out)

        flat_out = self.flat(iaf3_out)
        
        fc1_out = self.fc1(flat_out)
        iaf4_out = self.iaf4(fc1_out)
        fc2_out = self.fc2(iaf4_out)
        iaf5_out = self.iaf5(fc2_out)

        return iaf5_out

In [5]:
snn = SNN()

In [6]:
x = torch.randn((1, *input_shape))

print(f'DynapcnnLayer 0 [core 0]:                       ... {(x.shape)}')
con1_out = snn.conv1(x)
print(f'                                                conv1: {list(con1_out.shape)}')
iaf1_out = snn.iaf1(con1_out)
print(f'                                                iaf1: {list(iaf1_out.shape)}')
pool1_out = snn.pool1(iaf1_out)
print(f'                                                pool1: {list(pool1_out.shape)}')
pool1a_out = snn.pool1a(iaf1_out)
print(f'                                                pool1a: {list(pool1a_out.shape)}\n')

print(f'DynapcnnLayer 1 [core 1]:                       ... {list(pool1_out.shape)}')
conv2_out = snn.conv2(pool1_out)
print(f'                                                conv2: {list(conv2_out.shape)}')
iaf2_out = snn.iaf2(conv2_out)
print(f'                                                iaf2: {list(iaf2_out.shape)}')
pool2_out = snn.pool2(iaf2_out)
print(f'                                                pool2: {list(pool2_out.shape)}\n')

added = snn.adder(pool1a_out, pool2_out)

print(f'DynapcnnLayer 2 [core 2]:                       ... {list(added.shape)} [ Merge(pool1a, pool2) ]')
conv3_out = snn.conv3(added)
print(f'                                                conv3: {list(conv3_out.shape)}')
iaf3_out = snn.iaf3(conv3_out)
print(f'                                                iaf3: {list(iaf3_out.shape)}\n')

flat_out = snn.flat(iaf3_out)

print(f'DynapcnnLayer 3 [core ]:                        ... {list(flat_out.shape)}')
fc1_out = snn.fc1(flat_out)
print(f'                                                fc1: {list(fc1_out.shape)}')
iaf4_out = snn.iaf4(fc1_out)
print(f'                                                iaf4: {list(iaf4_out.shape)}\n')


print(f'DynapcnnLayer 4 [core ]:                        ... {list(iaf4_out.shape)}')
fc2_out = snn.fc2(iaf4_out)
print(f'                                                fc2: {list(fc2_out.shape)}')
iaf5_out = snn.iaf5(fc2_out)
print(f'                                                iaf5: {list(iaf5_out.shape)}\n')

DynapcnnLayer 0 [core 0]:                       ... torch.Size([1, 1, 28, 28])
                                                conv1: [1, 30, 27, 27]
                                                iaf1: [1, 30, 27, 27]
                                                pool1: [1, 30, 13, 13]
                                                pool1a: [1, 30, 6, 6]

DynapcnnLayer 1 [core 1]:                       ... [1, 30, 13, 13]
                                                conv2: [1, 30, 12, 12]
                                                iaf2: [1, 30, 12, 12]
                                                pool2: [1, 30, 6, 6]

DynapcnnLayer 2 [core 2]:                       ... [1, 30, 6, 6] [ Merge(pool1a, pool2) ]
                                                conv3: [1, 1, 4, 4]
                                                iaf3: [1, 1, 4, 4]

DynapcnnLayer 3 [core ]:                        ... [1, 16]
                                                fc1: [1, 500]
          

## DynapcnnNetwork

In [7]:
hw_model = DynapcnnNetworkGraph(
    snn,
    discretize=True,
    input_shape=input_shape
)

>  0
>  1
>  2
torch.Size([1, 16]) 16


ValueError: not enough values to unpack (expected 3, got 0)